
# Nonlinear problems (Unit 3.7)

We want to solve a nonlinear PDE.
## A simple scalar PDE

We consider the simple PDE
$$−\Delta u + 3 u^3 = 1 \text{ in } \Omega$$
on the unit square $\Omega =(0,1)^2$. We note that this PDE can also be formulated as a nonlinear minimization problem (cf. unit 3.8 ).

DD: not stated here but u is assumed to be zero on the boundary.

In [1]:
import netgen.gui
%gui tk
import tkinter
from netgen.geom2d import unit_square
from ngsolve import *

mesh = Mesh (unit_square.GenerateMesh(maxh=0.3))



In NGSolve we can solve the PDE conveniently using the linearization feature of SymbolicBFI.

The "BilinearForm" (which is not bilinear!) needed in the weak formulation is
$$A(u,v)=\int_Ω \nabla u \nabla v + 3 u^3 v - 1v dx \hspace{2em} (=0 \ \forall v \in H_0^1)$$

In [2]:
V = H1(mesh, order=3, dirichlet=[1,2,3,4])
u,v = V.TnT()
a = BilinearForm(V)
# Note: the updated tutorial had left the old function 5u^2 v
a += SymbolicBFI( grad(u) * grad(v) + 3*u**3*v - 1 * v)

### Newton's method

We use Newton's method and make the loop:

- Given an initial guess $u_0$

- loop over $i=0,\dots$ until convergence:

   - Compute linearization: $A(u^i)+\delta A(u^i)\Delta u^i=0$

       - $f^i=A(u^i)$
       - $B^i=\delta A(u^i)$
       - Solve $B^i\Delta u^i=−f^i$

    - Update $u^{i+1}=u^i+\Delta u^i$

    - Evaluate stopping criteria

- Evaluate $E(u^{i+1})$

As a stopping criteria we take $\langle Au^i,\Delta u^i\rangle=\langle Au^i,Au^i\rangle_{(B^i)^{−1}} < \epsilon$

.



In [3]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space # just so we don't have to pass it in explicitly
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec) # linearize at u^i, a.mat now represents delta A(u^i)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break    

In [4]:
gfu = GridFunction(V)
Draw(gfu,mesh,"u")
SimpleNewtonSolve(gfu,a)

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  0.18745474302207427
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  9.417689979256573e-05
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  8.541958907560332e-11
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  2.850975276745901e-17


## A trivial problem:

$$
  5 u^2 = 1, \qquad u \in \mathbb{R}.
$$

I think NumberSpace is just what it sounds like, a space of numbers (not functions) 

For more on NumberSpace, see [This forum thread](https://ngsolve.org/forum/ngspy-forum/86-solving-pde-with-non-trivial-null-space)

In [5]:
V = NumberSpace(mesh)
u,v = V.TnT()
a = BilinearForm(V)
a += SymbolicBFI( 5*u*u*v - 1 * v)
gfu = GridFunction(V)
gfu.vec[:] = 1
SimpleNewtonSolve(gfu,a)

print("\nscalar solution", gfu.vec[0], "(exact: ", sqrt(0.2), ")")

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  1.264911064067352
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.3265986323710903
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  0.04114755998989123
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  0.000857426926869178
Iteration   4  <A u 4 , A u 4 >_{-1}^0.5 =  3.8832745226099987e-07
Iteration   5  <A u 5 , A u 5 >_{-1}^0.5 =  7.979879233426057e-14

scalar solution 0.4472135954999579 (exact:  0.4472135954999579 )


## Another example: Stationary Navier-Stokes:
Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$, $\lambda \in \mathbb{R}$ so that
\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} + (\mathbf{u} \cdot \nabla) \mathbf{u} \cdot \mathbf{v}& - \int_{\Omega} \operatorname{div}(\mathbf{v}) p & &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q & & 
+ \int_{\Omega} \lambda q
&= 0 && \forall q \in Q, \\
& \int_{\Omega} \mu p & &= 0 && \forall \mu \in \mathbb{R}.
\end{align}


In [8]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.05))
V = VectorH1(mesh,order=3, dirichlet="bottom|right|top|left")
Q = H1(mesh,order=2)
N = NumberSpace(mesh)
X = FESpace([V,Q,N])
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
nu = Parameter(1)
# Note: grad(u) is a matrix where each row is the gradient of one of the components
a += SymbolicBFI(nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)-div(u)*q-div(v)*p-lam*q-mu*p)
gfu = GridFunction(X)
# Interpolate a velocity function along the top edge
gfu.components[0].Set(CoefficientFunction((4*x*(1-x),0)),definedon=mesh.Boundaries("top"))

In [9]:
SimpleNewtonSolve(gfu,a)
Draw(gfu.components[1],mesh,"p")
Draw(gfu.components[0],mesh,"u")

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  2.803070373102593
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.007950162870585377
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  6.526630117647601e-08
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  1.0046507136493525e-15


In [10]:
nu.Set(0.01)
SimpleNewtonSolve(gfu,a)
Redraw()

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  0.08811927351272117
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.008271023230528704
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  0.0001063894118075951
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  1.9401619253335152e-08
Iteration   4  <A u 4 , A u 4 >_{-1}^0.5 =  5.712580424841747e-16
